In [ ]:
import pandas as pd
df = pd.read_csv("data/types/sample_N50_d2_B5_G5_Utils1.csv")

In [19]:
import pandas as pd
df = pd.read_csv("finalsample.csv")

In [20]:
averages = df.mean(numeric_only=True)

# Print the averages
print(averages)

solveMILP                       9.481
solveSeqRLBucket                4.160
solveConicGibbsGreedyDynamic    9.948
dtype: float64


In [7]:

rl_col = 'solveSeqRLBucket'
exclude_cols = {rl_col, 'agents'}
other_columns = [col for col in df.columns if col not in exclude_cols]

# Calculate the percentage for each column
percentages = {
    col: (df[rl_col] > df[col]).sum() / len(df) * 100
    for col in other_columns
}

# Display the results
for col, percentage in percentages.items():
    print(f"{rl_col} > {col}: {percentage:.2f}% of rows")

solveSeqRLBucket > solveMILP: 10.10% of rows
